In [1]:
import pickle
import pandas as pd
#import tensorflow as tf
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Input, Dense, Concatenate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
usuaris = pd.read_pickle('../../data/casos.pkl')
llibres = pd.read_pickle('../../data/llibres.pkl')

In [3]:
# Dimensiones de los vectores de usuario y libro
dim_usuari = len(usuaris["vector"].iloc[0])
dim_llibre = len(llibres["vector"].iloc[0])
print("Dimensió vector usuari:", dim_usuari)
print("Dimensió vector llibre:", dim_llibre)

Dimensió vector usuari: 85
Dimensió vector llibre: 85


In [4]:
def vector_llibre(id_llibre):
    # Pasamos el book_id del libro y devolvemos su vector
    return llibres[llibres["book_id"] == int(id_llibre)]["vector"].iloc[0]

In [5]:
# Inicializa listas vacías para almacenar los datos
vectores_usuarios = []
vectores_libros = []
puntuaciones = []

for user in usuaris.itertuples():
    # Obtiene el vector del usuario
    vector_usuario = user.vector

    # Para cada libro leído y su puntuación
    for book_id, val in zip(user.llibres_usuari, user.val_llibres):
        # Obtiene el vector del libro (modifica esto según tu implementación)
        vector_libro = vector_llibre(book_id)

        # Añade los datos al conjunto de datos
        vectores_usuarios.append(vector_usuario)
        vectores_libros.append(vector_libro)
        puntuaciones.append(val)

    # Repite para los libros recomendados
    for book_id, punt in zip(user.llibres_recomanats, user.puntuacions_llibres):
        # Obtiene el vector del libro (modifica esto según tu implementación)
        vector_libro = vector_llibre(book_id)

        # Añade los datos al conjunto de datos
        vectores_usuarios.append(vector_usuario)
        vectores_libros.append(vector_libro)
        puntuaciones.append(punt)

# Convertir las listas en arrays de NumPy para su uso con TensorFlow/Keras
import numpy as np

vectores_usuarios_np = np.array(vectores_usuarios)
vectores_libros_np = np.array(vectores_libros)
puntuaciones_np = np.array(puntuaciones)

In [6]:
print(vectores_usuarios_np.shape)
print(vectores_libros_np.shape)
print(puntuaciones_np.shape)

(22794, 85)
(22794, 85)
(22794,)


In [7]:
X = np.concatenate([vectores_usuarios_np, vectores_libros_np], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, puntuaciones, test_size=0.2, random_state=42)

In [8]:
y_test = np.array(y_test)
def pre_pred(y_pred):
    y_pred_round = np.round(y_pred)
    y_pred_round = y_pred_round.astype(int)

    y_pred_round[y_pred_round > 5] = 5
    y_pred_round[y_pred_round < 1] = 1
    return y_pred_round

In [9]:
def precis(y_pred, y_test):
    # Definir un umbral de aceptación
    umbral = 0.5

    # Calcular el número de predicciones "correctas"
    predicciones_correctas = np.abs(y_pred.flatten() - y_test) <= umbral

    # Calcular la precisión
    precision = np.mean(predicciones_correctas)
    return precision

In [10]:
def print_metrics(pred, y_test):
    # Calculando el Mean Squared Error (MSE) y el Root Mean Squared Error (RMSE)
    mse = mean_squared_error(y_test, pred)

    # Calculando el Coeficiente de Determinación (R^2)
    r2 = r2_score(y_test, pred)

    print(f"Error Cuadrático Medio (MSE): {mse}")
    print(f"Coeficiente de Determinación (R^2): {r2}")
    precision = precis(pred, y_test)
    print(f'Precisión aproximada: {precision * 100:.2f}%')

In [11]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred_round = pre_pred(model.predict(X_test))

print(y_pred_round.shape)
print(y_test.shape)
for i in range(0, len(y_pred_round)):
    print(y_pred_round[i], y_test[i])

(4559,)
(4559,)
4 4
3 4
4 5
4 5
5 5
4 3
4 5
4 2
4 5
5 4
4 3
4 3
5 5
4 5
4 5
4 4
4 5
4 5
4 3
4 5
3 5
2 4
5 5
4 3
4 4
4 5
4 4
4 5
4 4
4 0
4 4
4 5
4 5
4 4
4 0
1 0
4 5
3 5
4 3
3 3
4 3
4 2
4 5
4 5
4 4
4 3
3 5
2 2
4 5
4 3
4 2
4 4
4 0
4 2
2 2
3 5
2 3
4 5
4 4
3 4
4 2
5 5
4 5
4 4
4 5
4 3
4 1
4 5
5 5
4 5
4 4
4 5
4 5
4 4
3 1
3 0
4 5
2 5
4 5
4 5
4 3
4 0
4 3
5 5
4 4
5 5
4 4
4 4
3 3
5 5
3 3
4 5
4 4
4 2
4 5
2 2
4 4
4 5
4 0
4 4
4 2
5 4
5 5
4 5
4 5
4 5
4 5
4 0
5 5
4 5
4 4
3 1
4 3
4 4
4 5
4 3
4 3
4 5
5 4
4 4
4 5
5 5
1 2
1 0
4 4
2 4
3 3
5 3
3 5
2 1
4 3
4 4
4 3
3 1
4 5
4 4
4 4
3 1
4 4
4 5
4 4
4 3
4 4
4 4
3 4
5 5
4 4
4 5
4 4
4 5
4 5
4 4
3 2
4 5
4 4
4 5
4 5
2 2
4 5
4 3
4 4
4 4
4 5
4 3
4 2
1 0
4 4
4 5
3 5
4 3
3 4
1 1
4 4
5 5
5 5
4 4
4 5
4 4
4 2
5 5
4 4
4 0
3 4
4 5
4 4
4 5
3 5
4 4
3 1
3 2
4 3
4 5
4 4
2 4
2 5
3 5
4 3
4 2
4 5
4 0
4 4
4 4
5 5
5 5
4 5
4 5
4 4
4 5
4 1
4 1
3 2
3 2
3 4
4 4
4 4
4 4
3 2
4 5
5 5
3 2
4 5
4 3
5 5
4 5
4 3
3 4
3 2
4 3
4 3
3 4
3 3
5 5
4 3
4 4
3 0
4 5
2 3
4 4
4 5
4 5
3 5
4 5
4 4
3 2
3 4
4 3


In [13]:
print_metrics(y_pred_round, y_test)

Error Cuadrático Medio (MSE): 1.4871682386488265
Coeficiente de Determinación (R^2): 0.16992724904454415
Precisión aproximada: 33.21%


In [14]:
# Crear el modelo base de Gradient Boosting Regressor
model = GradientBoostingRegressor(random_state=42)

# Definir la cuadrícula de parámetros a buscar
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

In [15]:
# Crear el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)

# Realizar la búsqueda
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=  10.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=  10.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=  10.6s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  20.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  20.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  20.5s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=  13.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=  13.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=  31.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=  31.6s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=  13.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_es

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error', verbose=2)

In [16]:
# Imprimir los mejores parámetros y el mejor score encontrado
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor MSE negativo:", grid_search.best_score_)

# Evaluar el mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_

Mejores parámetros: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Mejor MSE negativo: -1.2066878821411444


In [17]:
y_pred_round = pre_pred(best_model.predict(X_test))
print_metrics(y_pred_round, y_test)

Error Cuadrático Medio (MSE): 1.2963369159903488
Coeficiente de Determinación (R^2): 0.27644100912289904
Precisión aproximada: 37.11%
